In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from tensorflow.keras.models import load_model
import keras

In [2]:
sys.path.append('E:/Github/2023-24d-fai2-adsai-group-cv5/')

In [3]:
from src.models.model_work import ModelWork
from src.models.model_evaluation import iou, f1
from src.data.data_preprocessing import *

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5988397598922190281
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5826936832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16251346356298847801
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [4]:
preprocess = DataPipelineSetup()
model_work = ModelWork()



In [5]:
# Load your trained model
model = model_work.load_model('best_model_root_masks')  # replace 'your_model_name' with the name of your model

# Define the path to your images
image_folder = 'E:/Github/2023-24d-fai2-adsai-group-cv5/data/raw/train/train'  # replace with the actual path to your images

# Loop over your images
for image_file in os.listdir(image_folder):
    # Read the image
    image = cv2.imread(os.path.join(image_folder, image_file))

    # Predict the mask for the image
    predicted_mask = model_work.predict_image(image, model)

    # Now you can use the predicted_mask for further processing

OSError: No file or directory found at e:\Github\2023-24d-fai2-adsai-group-cv5\models\your_model_name.keras

In [ ]:
test = cv2.imread('../../Dataset/train_cropped/048_43-14-ROOT1-2023-08-08_pvd_OD0001_col-0_01-Fish Eye Corrected.png')
test = test /255.0

In [ ]:
model = tf.keras.models.load_model('../best_models/best_model_root.h5', custom_objects={'f1': f1, 'iou':iou})
def predict_image(img, patch_size):
    img = padder(img, patch_size)
    patches = patchify(img, (patch_size, patch_size, 3), step=patch_size)
    patch_x = patches.shape[0]
    patch_y = patches.shape[1]
    patches = patches.reshape(-1, patch_size, patch_size, 3)
    preds = model.predict(patches)
    preds = preds.reshape(patch_x, patch_y, patch_size, patch_size)
    predicted_mask = unpatchify(preds, (img.shape[0], img.shape[1]))
    predicted_mask = predicted_mask > 0.85
    plt.imshow(predicted_mask)
    print(np.max(preds))
    # iou_value = iou(K.variable(labels), K.variable(preds))
    # iou_value = K.eval(iou_value)
    # print("IoU:", iou_value)
    return predicted_mask, preds, patches

In [ ]:
predicted_mask, preds, patches=predict_image(test,256)

In [ ]:
def plot_iou(mask_image, prediction, patch_size=256):
    mask = cv2.imread(mask_image, 0)
    mask_pad = padder(mask, patch_size)
    reshaped_img = mask_pad.reshape((1,) + mask_pad.shape + (1,))
   
    binary_pred = prediction > 0.6
    pred = np.array(binary_pred, dtype=np.uint8)
    reshaped_pred = pred.reshape((1,) + pred.shape + (1,))
   
    reshaped_img = tf.cast(reshaped_img, dtype=tf.float32)
    reshaped_pred = tf.cast(reshaped_pred, dtype=tf.float32)
   
    iou_score = iou(reshaped_img, reshaped_pred)
   
    return iou_score

In [ ]:
threshold = 0.6

In [ ]:
# # Convert predicted_mask to np.uint8
predicted_mask_uint8 = (predicted_mask > threshold).astype(np.uint8) * 255


# Find connected components
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(predicted_mask_uint8)


In [ ]:
# Find connected components
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(predicted_mask_uint8)

# Set area and top edge position thresholds
area_threshold = 1200
top_threshold = 600


# Find the indices of the connected components based on area
area_indices = np.where(stats[1:, cv2.CC_STAT_AREA] >= area_threshold)[0] + 1

# Find the indices of the connected components based on top edge position
top_indices = np.where(stats[1:, cv2.CC_STAT_TOP] <= top_threshold)[0] + 1

# Take the intersection of the two sets of indices
filtered_label_indices = np.intersect1d(area_indices, top_indices)

# Sort the connected components based on their areas
sorted_indices = np.argsort(stats[filtered_label_indices, cv2.CC_STAT_AREA])

# Select the top 5 largest labels
top_5_indices = filtered_label_indices[sorted_indices[-5:]]

# Create a mask for the top 5 connected components
top_5_components_mask = np.isin(labels, top_5_indices).astype(np.uint8) * 255

# Display the original binary image, the largest connected components, and the filtered components
plt.figure(figsize=(80, 80))
plt.subplot(133), plt.imshow(top_5_components_mask)

plt.show()

In [ ]:
from skimage.morphology import skeletonize
skeleton = skeletonize(top_5_components_mask)
plt.imshow(skeleton, cmap='gray')
plt.title("Binary Mask")

In [ ]:
# Find connected components in the top 5 components mask
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(top_5_components_mask)

# Find the indices of the largest connected components
largest_label_indices = np.argsort(stats[1:, cv2.CC_STAT_AREA])[:5] + 1

# Create a mask for the largest connected components
largest_components_mask = np.isin(labels, largest_label_indices).astype(np.uint8) * 255

# Display the largest connected components
plt.figure(figsize=(30,30))
plt.subplot(122), plt.imshow(largest_components_mask)
plt.show()


In [ ]:
i = 0
for label in np.unique(labels):
    color = i + 1
    top_5_components_mask[labels == label] = color
    i+=1
    print(color)

In [ ]:
# Extract individual labels from the segmented image
unique_labels = np.unique(top_5_components_mask)

# Skip the background label (assuming it's 1)
unique_labels = unique_labels[1:]

# Create a new figure for subplots
fig, axs = plt.subplots(1, len(unique_labels), figsize=(15, 5))

# Iterate through each unique label and plot the corresponding plant
for i, label in enumerate(unique_labels):
    # Get the connected component index for the current label
    component_idx = label - 1  # Assuming labels start from 1

    # Get the bounding box for the label from 'stats'
    x, y, w, h = stats[component_idx, cv2.CC_STAT_LEFT], stats[component_idx, cv2.CC_STAT_TOP], \
                 stats[component_idx, cv2.CC_STAT_WIDTH], stats[component_idx, cv2.CC_STAT_HEIGHT]

    # Calculate the new bounding box dimensions to make it 1.5 times bigger
    scale_factor = 1.5
    new_w = int(w * scale_factor)
    new_h = int(h * scale_factor)

    # Calculate the position to center the new bounding box within the original bounding box
    x = max(0, x - int((new_w - w) / 2))
    y = max(0, y - int((new_h - h) / 2))

    # Ensure that the new bounding box coordinates are within the image boundaries
    new_w = min(new_w, top_5_components_mask.shape[1] - x)
    new_h = min(new_h, top_5_components_mask.shape[0] - y)

    # Crop the region of interest from the original image
    
    plant_roi = top_5_components_mask[y:y + new_h, x:x + new_w]

    # Save the cropped plant image
    output_path = f'../plants_cropped/plant_{7-label}.png'
    cv2.imwrite(output_path, plant_roi)

In [ ]:
for filename in os.listdir(images):
    if filename.endswith('.tif'):
        image_base = os.path.splitext(filename)[0]
        image_path = os.path.join(images, filename)
        image = cv2.imread(image_path, 0)
        image = image[:-15, 0:-100]
        edges = cv2.Canny(image, 0, 255)
        points = np.argwhere(edges > 0)
        
        y, x = points.min(axis=0)
        w, h = points.max(axis=0)
        
        range_y = w - y
        mid_x = (x + h) / 2

        x = int(mid_x - range_y / 2)
        h = int(mid_x + range_y / 2)

        roi = image[y:w,x:h]

        cv2.imwrite(f'./cropped/{filename}',roi)

In [ ]:

# Load the model outside the loop
model = tf.keras.models.load_model('../best_models/best_model_root.h5', custom_objects={'f1': f1, 'iou': iou})

# Specify folder path
folder_path = '../task_8_files/Kaggle Dataset/cropped/'

# Initialize the results list outside the loop
results = []
part_bounding_boxes = {}
# Loop through all image files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(('.tif')):
        file_path = os.path.join(folder_path, file_name)

        # Load the image
        img = cv2.imread(file_path)

        # Run the prediction function
        predicted_mask = predict_image(img, 256, model)
        # kernel = np.ones((4, 4), np.uint8)
        # predicted_mask = cv2.dilate(predicted_mask, kernel, iterations=1)
        # predicted_mask = cv2.erode(predicted_mask, kernel, iterations=1)

        # Apply connected components analysis and cropping code here
        predicted_mask_uint8 = (predicted_mask > 0.05).astype(np.uint8)
        preds = np.array(predicted_mask_uint8, dtype=np.uint8)

        # Find connected components
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(preds)

        # Sort components based on area in descending order
        sorted_components = sorted(range(1, retval), key=lambda x: stats[x, cv2.CC_STAT_AREA], reverse=True)

        # Define specific criteria
        min_area = 200
        min_top = 300
        max_top = 1000
        max_left = 2600

        # Select only components meeting specific criteria
        selected_components = [
            i for i in sorted_components if
            min_area <= stats[i, cv2.CC_STAT_AREA] and
            min_top <= stats[i, cv2.CC_STAT_TOP] <= max_top and
            max_left >= stats[i, cv2.CC_STAT_LEFT] <= max_left
        ]

        # Create an empty array for the filtered image with unique labels for each part
        filtered_image = np.zeros_like(labels, dtype=np.uint8)

        # Create a dictionary to store the bounding box with the maximum area for each part_number
        part_boxes = {}

        # Calculate the height of each part
        height, width, _ = img.shape
        part_width = (width // 5) - 30

        # If no components were selected, create results with length 0 for all parts
        if not selected_components:
            for part_number in range(1, 6):
                result_dict = {
                    'Plant ID': f"{os.path.splitext(file_name)[0]}_plant_{part_number}",
                    'Length (px)': 0
                }
                results.append(result_dict)
        else:
            # Draw bounding boxes with the maximum area for each part_number
            for idx, component_idx in enumerate(selected_components, start=1):
                x, y, w, h = stats[component_idx, cv2.CC_STAT_LEFT], stats[component_idx, cv2.CC_STAT_TOP], \
                            stats[component_idx, cv2.CC_STAT_WIDTH], stats[component_idx, cv2.CC_STAT_HEIGHT]

                # Calculate part number based on the y-coordinate
                part_number = min(x // part_width + 1, 5)

                # Update the bounding box if it has a larger area and is within the allowed region
                if part_number not in part_boxes or \
                        (w * h > part_boxes[part_number][2] * part_boxes[part_number][3] and
                         x >= (part_number - 1) * part_width and x <= part_number * part_width):
                    part_boxes[part_number] = ((x, y, w, h))

                filtered_image[labels == component_idx] = part_number
            # Loop through all selected components
            for idx, component_idx in enumerate(selected_components, start=1):
                x, y, w, h = stats[component_idx, cv2.CC_STAT_LEFT], stats[component_idx, cv2.CC_STAT_TOP], \
                            stats[component_idx, cv2.CC_STAT_WIDTH], stats[component_idx, cv2.CC_STAT_HEIGHT]

                # Calculate part number based on the y-coordinate
                part_number = min(x // part_width + 1, 5)
                # If the part_number is not in the dictionary, initialize the bounding box
                if part_number not in part_bounding_boxes:
                    part_bounding_boxes[part_number] = [x, y, x + w, y + h]
                else:
                    # Update bounding box if the part_number already exists in the dictionary
                    part_bounding_boxes[part_number][0] = min(part_bounding_boxes[part_number][0], x)
                    part_bounding_boxes[part_number][1] = min(part_bounding_boxes[part_number][1], y)
                    part_bounding_boxes[part_number][2] = max(part_bounding_boxes[part_number][2], x + w)
                    part_bounding_boxes[part_number][3] = max(part_bounding_boxes[part_number][3], y + h)

            # Draw merged bounding boxes and annotate with part numbers
            for part_number, bbox in part_bounding_boxes.items():
                x_min, y_min, x_max, y_max = bbox
                cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 0, 255), 3)
                cv2.putText(img, str(part_number), (x_min + (x_max - x_min) // 2 - 10, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 10)

            # Initialize a dictionary to store file names for each part_number
            file_names_by_part = {}

            # Draw bounding boxes with the maximum area for each part_number
            for part_number, box in part_boxes.items():
                
                x, y, w, h = box
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 3)
                cv2.putText(img, str(part_number), (x + w // 2 - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 10)

                # Store file name for each part_number
                file_names_by_part[part_number] = file_name

            # Crop bounding boxes on the filtered_image
            cropped_roots = []
            for part_number, box in part_boxes.items():
                x, y, w, h = box
                cropped_root = filtered_image[y:y+h, x:x+w]
                cropped_roots.append(cropped_root)
           
            # Skeletonize each cropped root
            skeletonized_roots = [skeletonize(root > 0) for root in cropped_roots]
            
            # Calculate the length of each main root and append to the results list
            for part_number in range(1, 6):
                if part_number <= len(skeletonized_roots) and part_number in part_boxes:
                    # Create subplot
                    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
                    
                    skeleton = skeletonized_roots[part_number - 1]
                    test = summarize(Skeleton(skeleton))
                    length = calculate_length(skeleton, test)
                    
                    ax.imshow(skeleton, cmap='gray')
                    # Plot circle for the end of the main root
                    main_max = test['node-id-dst'].idxmax()
                    x_coord_dst = test.loc[main_max, 'image-coord-dst-1']
                    y_coord_dst = test.loc[main_max, 'image-coord-dst-0']
                    ax.add_patch(plt.Circle((x_coord_dst, y_coord_dst), radius=10, color='gray', fill=False))

                    # Plot circle for the beginning of the main root
                    main_min = test['node-id-src'].idxmin()
                    x_coord_src = test.loc[main_min, 'image-coord-src-1']
                    y_coord_src = test.loc[main_min, 'image-coord-src-0']
                    ax.add_patch(plt.Circle((x_coord_src, y_coord_src), radius=10, color='green', fill=False))
                    ax.set_aspect('equal', 'box')
                    plt.tight_layout()
                    plt.show()
                else:
                    length = 0

                file_name_for_part = os.path.splitext(file_names_by_part.get(part_number, file_name))[0]  # Remove file extension

                # Append results to the list
                result_dict = {
                    'Plant ID': f"{file_name_for_part}_plant_{part_number}",
                    'Length (px)': length
                }
                results.append(result_dict)

                print(f"Root Length for Plant {part_number}: {length}")
                
# Create a DataFrame with columns "Plant ID" and "Length (px)"
df = pd.DataFrame(results)

# Check if the CSV file already exists
csv_filename = '../task_8_files/Kaggle Dataset/kaggle.csv'
file_exists = os.path.exists(csv_filename)

# Save the DataFrame to a CSV file
df.to_csv(csv_filename, index=False, mode='a', header=not file_exists)

print(f"Results saved to {csv_filename}")